# Projeto Final de Machine Learning
Feito por: _Henrique Bucci_ e _Marcelo Alonso_

Dados e informações: https://www.kaggle.com/datasets/marcopale/housing/data

**Perguntas**  
- Posso remover o PID?
    - **R:** Sim
- Posso criar colunas a partir de contas de outras antes de fazer a seleção?
    - **R:** Sim
- Se eu aplicar PolynomialFeatures nos dados, eles também contam como features para a contagem?
    - **R:** Fazer PolyFeatures depois de selecionar as features
- Posso utilizar correlação na análise exploratória?
    - **R:** Pode, mas é "inútil"
- Posso utilizar métodos de clustering na pipeline para incluir a classificação como uma nova feature?
    - **R:** SoftMax no resultado do Kmeans para exagerar a classe mais próxima.
- Posso utilizar algum método de Dimensionality Reduction (ex: PCA) para me ajudar a escolher as features?
    - **R:** Sim.


Testar stacking: Treinar diversos modelos e treinar um modelo final com os predicts destes modelos.

#### ANOTAÇÕES
Utilizar LASSO para seleção de features.

Regressão linear para ignorar outliers.

RANSAC -> regressao linear que ignora outliers

## Etapa 0

Nesta etapa, iremos:
- Importar bibliotecas
- Carregar os dados
- Verificar se existem colunas que não fazem sentido serem colocadas no dataset final (como ID ou algum outro tipo de identificador arbitrário), olhando apenas a descrição das colunas.
- Separar o dataset em Treino-Teste

### Bibliotecas

In [1]:
import pandas as pd
from datetime import datetime
from utils import *


from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

### Constantes

In [2]:
SEED = 420

### Carregamento e Pré-processamento dos Dados

In [3]:
dataset = load_data()
dataset.head()

,Order,PID,MS.SubClass,MS.Zoning,Lot.Frontage,Lot.Area,Street,Alley,Lot.Shape,Land.Contour,...,Pool.Area,Pool.QC,Fence,Misc.Feature,Misc.Val,Mo.Sold,Yr.Sold,Sale.Type,Sale.Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [4]:
"""
Neste caso, a presença de duplicatas não seria intencional, uma vez que cada casa deveria ser única.
Portanto, vamos removê-las.
"""
print(f"Total de linhas antes de remover duplicatas: {dataset.shape[0]}")
dataset.drop_duplicates(inplace=True) # Removendo duplicatas
print(f"Total de linhas depois de remover duplicatas: {dataset.shape[0]}")

Total de linhas antes de remover duplicatas: 2930
Total de linhas depois de remover duplicatas: 2930


In [5]:
"""
Como a primeira coluna é o ID da observação e a segunda é um identificador, podemos removê-las, uma vez que estes são valores arbitrários.
"""
dataset = dataset.iloc[:, 2:] # Estamos removendo as duas primeiras colunas, que são o ID e o PID (Parcel identification number)

### Criando novas features

Ao analisarmos as features da forma descrita acima, vimos espaço para a criação de novas features que podem ser úteis na modelagem dos dados:
- **Tot Lot Area** : `Lot Frontage + Lot Area`
- **Bsmt Tot Bath** : `Bsmt Full Bath + 0.5*Bsmt Half Bath`
- **Garage Area/Car** : `Garage Area / Garage Cars`
- **Tot Porch SF** : `Open Porch SF + Enclosed Porch + 3Ssn Porch + Screen Porch`
- **Date Sold** : `timestamp(Month Sold, Year Sold)`

In [6]:
dataset.loc[:, 'Tot.Lot.Area'] = dataset.loc[:, 'Lot.Frontage'] + dataset.loc[:, 'Lot.Area']
dataset.loc[:, 'Bsmt.Tot.Bath'] = dataset.loc[:, 'Bsmt.Full.Bath'] + 0.5*dataset.loc[:, 'Bsmt.Half.Bath']
dataset.loc[:, 'Garage.Area/Cars'] = dataset.loc[:, 'Garage.Area'] / dataset.loc[:, 'Garage.Cars']
dataset.loc[:, 'Tot.Porch.SF'] = dataset.loc[:, 'Open.Porch.SF'] + dataset.loc[:, 'X3Ssn.Porch'] + dataset.loc[:, 'Enclosed.Porch'] + dataset.loc[:, 'Screen.Porch']
dataset.loc[:, 'Date.Sold'] = pd.to_datetime(dict(year=dataset['Yr.Sold'], month=dataset['Mo.Sold'], day=1)).apply(lambda x: x.timestamp())

### Train-Test Split

- A partir de agora, usaremos apenas o dataset de treino, a partição de teste será tratada como se não existisse ainda.
- O dataset total será dividido em uma proporção 80/20, uma vez que temos poucos dados (2930 no total).
- Por não se tratar de uma série temporal, podemos aplicar uma aleatoriedade na partição.

In [7]:
X, y = dataset.drop('SalePrice', axis=1), dataset.loc[:, 'SalePrice']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

## Etapa 1

### Análise Exploratória

Nesta parte, iremos fazer uma análise global dos dados, apenas para garantir a integridade destes.  
Assim sendo, iremos procurar entender quais são as features e target, quais são seus respectivos tipos e buscar outras informações como:
- Dados nulos
- Dados duplicados
- Outliers
- Spikes
- Erros grosseiros

Além disso, iremos buscar saber a distribuição e a "cara" de cada variável.

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 85 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MS.SubClass       2930 non-null   int64  
 1   MS.Zoning         2930 non-null   object 
 2   Lot.Frontage      2440 non-null   float64
 3   Lot.Area          2930 non-null   int64  
 4   Street            2930 non-null   object 
 5   Alley             198 non-null    object 
 6   Lot.Shape         2930 non-null   object 
 7   Land.Contour      2930 non-null   object 
 8   Utilities         2930 non-null   object 
 9   Lot.Config        2930 non-null   object 
 10  Land.Slope        2930 non-null   object 
 11  Neighborhood      2930 non-null   object 
 12  Condition.1       2930 non-null   object 
 13  Condition.2       2930 non-null   object 
 14  Bldg.Type         2930 non-null   object 
 15  House.Style       2930 non-null   object 
 16  Overall.Qual      2930 non-null   int64  


In [10]:
dataset.describe(include='all')

,MS.SubClass,MS.Zoning,Lot.Frontage,Lot.Area,Street,Alley,Lot.Shape,Land.Contour,Utilities,Lot.Config,...,Mo.Sold,Yr.Sold,Sale.Type,Sale.Condition,SalePrice,Tot.Lot.Area,Bsmt.Tot.Bath,Garage.Area/Cars,Tot.Porch.SF,Date.Sold
count,2930.000000,2930,2440.000000,2930.000000,2930,198,2930,2930,2930,2930,...,2930.000000,2930.000000,2930,2930,2930.000000,2440.000000,2928.000000,2772.000000,2930.000000,2.930000e+03
unique,NaN,7,NaN,NaN,2,2,4,4,3,5,...,NaN,NaN,10,6,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,RL,NaN,NaN,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,NaN,NaN,WD,Normal,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,2273,NaN,NaN,2918,120,1859,2633,2927,2140,...,NaN,NaN,2536,2413,NaN,NaN,NaN,NaN,NaN,NaN
mean,57.387372,NaN,69.224590,10147.921843,NaN,NaN,NaN,NaN,NaN,NaN,...,6.216041,2007.790444,NaN,NaN,180796.060068,9778.747541,0.461919,272.233574,89.139590,1.206200e+09
std,42.638025,NaN,23.365335,7880.017759,NaN,NaN,NaN,NaN,NaN,NaN,...,2.714492,1.316613,NaN,NaN,79886.692357,6435.636376,0.520835,61.659127,107.734138,4.105835e+07
min,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,2006.000000,NaN,NaN,12789.000000,1324.000000,0.000000,100.000000,0.000000,1.136074e+09
25%,20.000000,NaN,58.000000,7440.250000,NaN,NaN,NaN,NaN,NaN,NaN,...,4.000000,2007.000000,NaN,NaN,129500.000000,7283.750000,0.000000,235.500000,0.000000,1.172707e+09
50%,50.000000,NaN,68.000000,9436.500000,NaN,NaN,NaN,NaN,NaN,NaN,...,6.000000,2008.000000,NaN,NaN,160000.000000,9331.500000,0.000000,264.000000,50.000000,1.207008e+09
75%,70.000000,NaN,80.000000,11555.250000,NaN,NaN,NaN,NaN,NaN,NaN,...,8.000000,2009.000000,NaN,NaN,213500.000000,11287.250000,1.000000,296.000000,136.000000,1.243814e+09


In [11]:
plot_Xtrain(X_train, 'x_train_original.png')

Gráfico salvo em ./graphs/x_train_original.png


In [12]:
# Agora, basta remover os outliers encontrados no target do dataset
X_train, y_train = remove_outliers(X_train, y_train)

In [13]:
cauda_direita = ['Lot.Frontage', 'Lot.Area', 'Mas.Vnr.Area', 'BsmtFin.SF.1', 'BsmtFin.SF.2',
                 'Bsmt.Unf.SF', 'Total.Bsmt.SF', 'X1st.Flr.SF', 'X2nd.Flr.SF', 'Gr.Liv.Area', 
                 'Garage.Area', 'Wood.Deck.SF', 'Open.Porch.SF', 'Enclosed.Porch', 
                 'Screen.Porch', 'X3Ssn.Porch', 'Tot.Lot.Area', 'Garage.Area/Cars', 'Tot.Porch.SF']

categorical = X.select_dtypes(include='object').columns.tolist()
numerical = X.select_dtypes(include='number').columns.tolist()

In [14]:
X_train_log = X_train.copy()
X_train_log[cauda_direita] = np.log1p(X_train_log[cauda_direita])

plot_Xtrain(X_train[numerical], 'x_train_log.png')


Gráfico salvo em ./graphs/x_train_log.png


In [15]:
num_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
])

c_log_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('log', FunctionTransformer(np.log1p, validate=False, feature_names_out='one-to-one')),
])

cat_pipe = Pipeline(steps=[
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

preprocessing_pipeline = ColumnTransformer(
    transformers = [
        ('num', num_pipe, [col for col in numerical if col not in cauda_direita]),
        ('clog', c_log_pipe, cauda_direita),
        ('cat', cat_pipe, categorical)
    ],
    remainder='passthrough'
)

preprocessing_pipeline

ColumnTransformer(remainder='passthrough',
                  transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['MS.SubClass', 'Overall.Qual', 'Overall.Cond',
                                  'Year.Built', 'Year.Remod.Add',
                                  'Low.Qual.Fin.SF', 'Bsmt.Full.Bath',
                                  'Bsmt.Half.Bath', 'Full.Bath', 'Half.Bath',
                                  'Bedroom.AbvGr', 'Kitchen.AbvGr',
                                  'TotRms.AbvGrd', 'Fireplaces',
                                  'Garage.Yr.Blt', 'Garage.Cars'...
                                  'Land.Contour', 'Utilities', 'Lot.Config',
                                  'Land.Slope', 'Neighborhood', 'Condition.1',
                                  'Condition.2', 'Bldg.Type', 'House.Style',
                                  'Roof.Style', 'Roof.Matl', 'Exterior.1st',
                                  'Exterior.2nd', 'Mas.Vnr.Type', 'Exter.Qual',
                                  'Exter.Cond', 'Foundation', 'Bsmt.Qual',
                                  'Bsmt.Cond', 'Bsmt.Exposure',
                                  'BsmtFin.Type.1', 'BsmtFin.Type.2', 'Heating',
                                  'Heating.QC', 'Central.Air', 'Electrical', ...])])

In [16]:
X_train_transformed = preprocessing_pipeline.fit_transform(X_train)

In [17]:
rnd_clf = RandomForestRegressor(
    n_estimators=700,
    max_leaf_nodes=16,
    random_state=SEED,
    n_jobs=1
)

In [18]:
rnd_clf.fit(X_train_transformed, y_train)

RandomForestRegressor(max_leaf_nodes=16, n_estimators=700, n_jobs=1,
                      random_state=420)

In [19]:
importances = rnd_clf.feature_importances_

feature_names = preprocessing_pipeline.get_feature_names_out()
feature_importances_df = pd.DataFrame(zip(feature_names, importances), columns=['Feature', 'Importance']) \
    .sort_values(by='Importance', ascending=False)

feature_importances_df = aggregate_categorical_importances(feature_importances_df.set_index('Feature'))


X_feats = [feat.split('__')[1] for feat in feature_importances_df[:15].index]

In [22]:
X_train = X_train[X_feats]

## Parte 3/4/5/6/...

1. Escolher modelos (métodos de stacking inclusos)
    - DummyRegressor
    - LinearRegression
    - Outros modelos básicos
        - Polynomial Features
        - Scalers
    - Pipelines avançadas
        - Utilizar KMeans como fonte de novas features na pipeline
        - Métodos de Ensemble
        
2. Montar GridSearchCV com hiperparâmetros

## Parte 5

Seleção de modelos com GridSearchCV

In [20]:
param_grid = [{
    'regressor' : [LinearRegression(), DummyRegressor()],
}, {
    'regressor': [Lasso(), Ridge()],
    'alpha': [0.1, 1, 10, 100],
}, {
    'regressor': [ElasticNet()],
    'alpha': [0.1, 1, 10, 100],
    'l1_ratio': [0.1, 0.5, 0.9]
}, {
    'regressor': [RandomForestRegressor()],
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}, {
    'regressor': [GradientBoostingRegressor()],
    'n_estimators': [10, 50, 100],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}]